In [21]:
import pandas as pd
import os

# Caminho da pasta com arquivos .xls
caminho_pasta = r"dados\passageiros"
saida=r"dados\passageiros_parquet"

def listar_todos_arquivos(diretorio_raiz, extensao=None):
    arquivos_encontrados = []
    for raiz, _, arquivos in os.walk(diretorio_raiz):
        for arquivo in arquivos:
            if extensao:
                if arquivo.lower().endswith(extensao.lower()):
                    arquivos_encontrados.append(os.path.join(raiz, arquivo))
            else:
                arquivos_encontrados.append(os.path.join(raiz, arquivo))
    return arquivos_encontrados

In [16]:
arquivos = listar_todos_arquivos(caminho_pasta,extensao='xls')

In [17]:
len(arquivos)

3851

In [18]:
# transforma os xls em parquet e unifica, para otimizar
dfs=[]

for caminho_arquivo in arquivos:
    try:

        # Lê o arquivo com pandas
        try: df_pd = pd.read_excel(caminho_arquivo,dtype=str)
        except: df_pd = pd.read_excel(caminho_arquivo)

        if not 'linha' in " ".join([x.lower() for x in df_pd.columns]):
            # Encontra a linha que contém "linha" para usar como cabeçalho
            idx_cabecalho = df_pd.apply(lambda row: row.astype(str).str.lower().str.contains("linha", case=False).any(), axis=1).idxmax()
            colunas = df_pd.iloc[idx_cabecalho].astype(str).str.strip().str.lower().tolist()

            # Lê os dados a partir da linha seguinte ao cabeçalho
            df_dados = df_pd.iloc[idx_cabecalho + 1:].reset_index(drop=True)
            df_dados.columns = colunas
        else:
            df_dados=df_pd

        df_dados['arquivo']=caminho_arquivo
        numericas=[x for x in df_dados.columns if "passag" in x.lower()]
        for n in numericas: df_dados[n]=df_dados[n].astype(int)
        dfs.append(df_dados)
        nome_arquivo=caminho_arquivo.split("\\")[-1].split('.')[0]
        print(nome_arquivo)
        caminho_parquet = os.path.join(saida, rf"{nome_arquivo}.parquet")
        df_dados.fillna("").to_parquet(caminho_parquet, index=False)
    except Exception as e: print("Erro em ",caminho_arquivo,e)


Passag-20150101
Passag-20150102
Passag-20150103
Passag-20150104
Passag-20150105
Passag-20150106
Passag-20150107
Passag-20150108
Passag-20150109
Passag-20150110
Passag-20150111
Passag-20150112
Passag-20150113
Passag-20150114
Passag-20150115
Passag-20150116
Passag-20150117
Passag-20150118
Passag-20150119
Passag-20150120
Passag-20150121
Passag-20150122
Passag-20150123
Passag-20150124
Passag-20150125
Passag-20150126
Passag-20150127
Passag-20150128
Passag-20150129
Passag-20150130
Passag-20150131
Passag-20150201
Passag-20150202
Passag-20150203
Passag-20150204
Passag-20150205
Passag-20150206
Passag-20150207
Passag-20150208
Passag-20150209
Passag-20150210
Passag-20150211
Passag-20150212
Passag-20150213
Passag-20150214
Passag-20150215
Passag-20150216
Passag-20150217
Passag-20150218
Passag-20150219
Passag-20150220
Passag-20150221
Passag-20150222
Passag-20150223
Passag-20150224
Passag-20150225
Passag-20150226
Passag-20150227
Passag-20150228
Passag-20150301
Passag-20150302
Passag-20150303
Passag-2

In [24]:
#le parquets para pegar as colunas únicas e começar as transformações
# Lista de arquivos .parquet na pasta
arquivos_parquet = listar_todos_arquivos(saida,extensao='parquet')
# print(len(arquivos_parquet), len(arquivos), len(dfs))
arquivos_parquet


['dados\\passageiros_parquet\\01-FEV.parquet',
 'dados\\passageiros_parquet\\01012019.parquet',
 'dados\\passageiros_parquet\\01022018.parquet',
 'dados\\passageiros_parquet\\01032018.parquet',
 'dados\\passageiros_parquet\\01042018.parquet',
 'dados\\passageiros_parquet\\01052018.parquet',
 'dados\\passageiros_parquet\\01072018.parquet',
 'dados\\passageiros_parquet\\010818.parquet',
 'dados\\passageiros_parquet\\01102018_Dados para Transparência.parquet',
 'dados\\passageiros_parquet\\01112018_Dados para Transparência.parquet',
 'dados\\passageiros_parquet\\01122018_Dados para Transparência.parquet',
 'dados\\passageiros_parquet\\01ABR2019.parquet',
 'dados\\passageiros_parquet\\01ABR2020.parquet',
 'dados\\passageiros_parquet\\01ABR2021(8).parquet',
 'dados\\passageiros_parquet\\01ABR2022.parquet',
 'dados\\passageiros_parquet\\01ABR2023(1).parquet',
 'dados\\passageiros_parquet\\01ABR2024.parquet',
 'dados\\passageiros_parquet\\01AGO2019.parquet',
 'dados\\passageiros_parquet\\01AG

In [29]:
# Lista para armazenar as colunas de cada dataframe
colunas_df = []
dfs=[]

# Lê os arquivos .parquet e armazena as colunas de cada dataframe
for caminho_arquivo in arquivos_parquet:
    try:
        # caminho_arquivo = os.path.join(saida, arquivo)
        # Lê o parquet no pandas
        df = pd.read_parquet(caminho_arquivo)
        df.columns=[x.lower() for x in df.columns]
        colunas_df.append([x.lower() for x in df.columns])
        numericas=[x for x in df.columns if "passag" in x.lower()]
        for n in numericas: df[n]=df[n].astype(int,errors='ignore')
        df['data']=pd.to_datetime(df['data'])
        dfs.append(df)
    except: print(df['arquivo'].to_list()[0])

#para visualização das colunas únicas
colunas_df = [list(x) for x in set(tuple(x) for x in colunas_df)]
lista=[]
for x in colunas_df: lista+= x

lista=list(set(lista))
lista.sort()
print("Colunas únicas:" , lista)

C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['data']=pd.to_datetime(df['data'])
C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['data']=pd.to_datetime(df['data'])
C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['data']=pd.to_datetime(df['data'])
C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a 

dados\passageiros\2018\Abril 2018_Consolidado.xls
dados\passageiros\2017\Consolidade Julho 2017.xls
dados\passageiros\2017\Consolidade Mes Setembro1.xls
dados\passageiros\2021\Consolidado 01-Janeiro-2021.xls
dados\passageiros\2022\Consolidado 01-Janeiro-2022.xls
dados\passageiros\2023\Consolidado 01-Janeiro-2023.xls
dados\passageiros\2024\Consolidado 01-Janeiro-2024.xls
dados\passageiros\2021\Consolidado 02-Fevereiro-2021.xls


C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['data']=pd.to_datetime(df['data'])
C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['data']=pd.to_datetime(df['data'])


dados\passageiros\2022\Consolidado 02-Fevereiro-2022.xls
dados\passageiros\2023\Consolidado 02-Fevereiro-2023.xls
dados\passageiros\2024\Consolidado 02-Fevereiro-2024.xls
dados\passageiros\2021\Consolidado 03-Março-2021.xls
dados\passageiros\2022\Consolidado 03-Março-2022.xls
dados\passageiros\2023\Consolidado 03-Março-2023.xls
dados\passageiros\2024\Consolidado 03-Março-2024.xls


C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['data']=pd.to_datetime(df['data'])
C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['data']=pd.to_datetime(df['data'])


...

dados\passageiros\2019\Consolidado Janeiro 2019.xls
dados\passageiros\2020\Consolidado Janeiro-2020.xls
dados\passageiros\2019\Consolidado Julho-2019.xls
dados\passageiros\2020\Consolidado Julho-2020.xls
dados\passageiros\2018\Consolidado Junho 2018(1).xls
dados\passageiros\2019\Consolidado Junho-2019.xls
dados\passageiros\2020\Consolidado Junho-2020.xls


C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['data']=pd.to_datetime(df['data'])
C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['data']=pd.to_datetime(df['data'])
C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['data']=pd.to_datetime(df['data'])
C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Could not infer format, so each element will be

dados\passageiros\2019\Consolidado Maio-2019.xls
dados\passageiros\2020\Consolidado Maio-2020.xls
dados\passageiros\2019\Consolidado Março 2019.xls
dados\passageiros\2020\Consolidado Março-2020.xls
dados\passageiros\2017\Consolidado Novembro 2017_Final2.xls
dados\passageiros\2018\Consolidado Novembro 2018.xls
dados\passageiros\2019\Consolidado Novembro-2019.xls
dados\passageiros\2020\Consolidado Novembro-2020.xls
dados\passageiros\2022\Consolidado Out-2022.xls
dados\passageiros\2018\Consolidado Outubro 2018.xls


C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['data']=pd.to_datetime(df['data'])
C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['data']=pd.to_datetime(df['data'])
C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['data']=pd.to_datetime(df['data'])
C:\Users\franc\AppData\Local\Temp\ipykernel_32744\349151083.py:15: UserWarning: Could not infer format, so each element will be

dados\passageiros\2019\Consolidado Outubro-2019(1).xls
dados\passageiros\2020\Consolidado Outubro-2020.xls
dados\passageiros\2017\Consolidado Outubro_Final1.xls
dados\passageiros\2019\Consolidado Setembro-2019.xls
dados\passageiros\2020\Consolidado Setembro-2020.xls
dados\passageiros\2018\Consolidado_Mar_2018_VF.xls
dados\passageiros\2018\Fevereiro 2018_Consolidado.xls
dados\passageiros\2018\Janeiro 2018 Consolidado.xls
dados\passageiros\2017\Junho Consolidado.xls
dados\passageiros\2017\Maio 2017 Consolidado_01.xls
dados\passageiros\2015\Passag-20150302.xls
dados\passageiros\2015\Pass_Transp_Abr15.xls
dados\passageiros\2016\Pass_Transp_Abr16.xls
dados\passageiros\2017\Pass_Transp_Abr17.xls
dados\passageiros\2015\Pass_Transp_Ago15.xls
dados\passageiros\2016\Pass_Transp_Ago16.xls
dados\passageiros\2015\Pass_Transp_Dez15.xls
dados\passageiros\2016\Pass_Transp_Dez16.xls
dados\passageiros\2015\Pass_Transp_Fev15.xls
dados\passageiros\2016\Pass_Transp_Fev16.xls
dados\passageiros\2017\Pass_Tra

In [31]:
renaming_dict={'area':"area",
 'arquivo':"arquivo",
 'data':"data",
 'empresa':"empresa",
 'grupo':"grupo",
 'linha':"linha",
 'lote':"lote",
 'passageiros com gratuidade':"passageiros_gratuitos",
 'passageiros com gratuidade\n\n(l)':"passageiros_gratuitos",
 'passageiros com gratuidade est':"passageiros_gratuitos_estudante",
 'passageiros com gratuidade estudante\n\n(m)':"passageiros_gratuitos_estudante",
 'passageiros com gratuidade estudante\n\n(n)':"passageiros_gratuitos_estudante",
 'passageiros comum e vt':"passageiros_comum_vt",
 'passageiros int ônibus->ônibus':"passageiros_integracao_onibus",
 'passageiros integrações ônibus -> ônibus\n\n(k)':"passageiros_integracao_onibus",
 'passageiros pagantes':"passageiros_pagantes",
 'passageiros pagantes\n\n(j) = (a+b+c+d+e+f+g+h+i)':"passageiros_pagantes",
 'passageiros pagantes bilhete único comum\n\n(b)':"passageiros_comum",
 'passageiros pagantes bilhete único comum mensal\n\n(c)':"passageiros_comum_mensal",
 'passageiros pagantes bilhete único estudante\n\n(d) ':"passageiros_estudante",
 'passageiros pagantes bilhete único estudante mensal\n\n(e) ':"passageiros_estudante_mensal",
 'passageiros pagantes bilhete único vale transporte\n\n(f)':"passageiros_vale_transporte",
 'passageiros pagantes bilhete único vale transporte mensal\n\n(g)':"passageiros_vale_transporte_mensal",
 'passageiros pagantes em dinheiro\n\n(a)':"passageiros_dinheiro",
 'passageiros pagantes integrações metrô e cptm\n\n(h)':"passageiros_integracao_metro_cptm",
 'passageiros pagantes integrações metrô e cptm mensal\n\n(i)':"passageiros_integracao_metro_cptm_mensal",
 'passageiros pagtes bu vt':"passageiros_comum_vt",
 'passageiros pagtes comum':"passageiros_comum",
 'passageiros pagtes em dinheiro':"passageiros_dinheiro",
 'passageiros pagtes estudante':"passageiros_estudante",
 'passageiros pagtes int m/cptm':"passageiros_integracao_metro_cptm",
 'passageiros pgts bu comum m':"passageiros_comum_mensal",
 'passageiros pgts bu est mensal':"passageiros_estudante_mensal",
 'passageiros pgts bu vt mensal':"passageiros_vale_transporte_mensal",
 'passageiros pgts int m/cptm m':"passageiros_integracao_metro_cptm_mensal",
 'soma de passageiros com gratuidade\n\n(l)':"passageiros_gratuitos",
 'soma de passageiros com gratuidade estudante\n\n(m)':"passageiros_gratuitos_estudante",
 'soma de passageiros integrações ônibus -> ônibus\n\n(k)':"passageiros_integracao_onibus",
 'soma de passageiros pagantes\n\n(j) = (a+b+c+d+e+f+g+h+i)':"passageiros_pagantes",
 'soma de passageiros pagantes bilhete único comum\n\n(b)':"passageiros_comum",
 'soma de passageiros pagantes bilhete único comum mensal\n\n(c)':"passageiros_comum_mensal",
 'soma de passageiros pagantes bilhete único estudante\n\n(d) ':"passageiros_estudante",
 'soma de passageiros pagantes bilhete único estudante mensal\n\n(e) ':"passageiros_estudante_mensal",
 'soma de passageiros pagantes bilhete único vale transporte\n\n(f)':"passageiros_vale_transporte",
 'soma de passageiros pagantes bilhete único vale transporte mensal\n\n(g)':"passageiros_vale_transporte_mensal",
 'soma de passageiros pagantes em dinheiro\n\n(a)':"passageiros_dinheiro",
 'soma de passageiros pagantes integrações metrô e cptm\n\n(h)':"passageiros_integracao_metro_cptm",
 'soma de passageiros pagantes integrações metrô e cptm mensal\n\n(i)':"passageiros_integracao_metro_cptm_mensal",
 'soma de total passageiros transportados\n\n(n) = (j + k + l + m)':'passageiros_total',
 'tipo':"tipo",
 'tot passageiros transportados':'passageiros_total',
 'total passageiros transportados\n\n(m) = (j + k + l )':'passageiros_total',
 'total passageiros transportados\n\n(m) = (j + k + l)':'passageiros_total',
 'total passageiros transportados\n\n(m) = (j + k + l+n)':'passageiros_total',
 'total passageiros transportados\n\n(n) = (j + k + l + m)':'passageiros_total'
 }

In [32]:
df_unificado=pd.DataFrame()
for df in dfs:
    df.columns=[x.lower() for x in df.columns]
    df=df.rename(columns=renaming_dict)
    df_unificado=pd.concat([df_unificado,df])

In [33]:
df_unificado

,data,tipo,area,empresa,linha,passageiros_dinheiro,passageiros_comum,passageiros_comum_mensal,passageiros_estudante,passageiros_estudante_mensal,...,passageiros_integracao_metro_cptm_mensal,passageiros_pagantes,passageiros_integracao_onibus,passageiros_gratuitos,passageiros_gratuitos_estudante,passageiros_total,arquivo,grupo,lote,passageiros_vale_transporte
0,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,N10511 - TERM CACHOEIRINHA/TERM LAPA,28,48.0,9,2,0,...,0.0,142,162,27,0.0,331,dados\passageiros\2019\01-FEV.xls,NaN,NaN,NaN
1,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,N10611 - T PQ D PEDRO /MET BARRA FUNDA,0,3.0,0,0,0,...,0.0,4,1,0,0.0,5,dados\passageiros\2019\01-FEV.xls,NaN,NaN,NaN
2,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,N14311 - METRO BARRA FUNDA/MORRO GRANDE,4,8.0,2,0,0,...,0.0,40,20,7,0.0,67,dados\passageiros\2019\01-FEV.xls,NaN,NaN,NaN
3,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,800001 - TERM LAPA/PCA RAMOS DE AZEVEDO,473,1290.0,208,35,0,...,1.0,3275,2541,1526,41.0,7383,dados\passageiros\2019\01-FEV.xls,NaN,NaN,NaN
4,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,830001 - TERM PIRITUBA/TERM LAPA,47,169.0,24,4,0,...,0.0,410,381,187,11.0,989,dados\passageiros\2019\01-FEV.xls,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1405,2017-01-31,PERMISSAO,AREA 8,TRANSCAP,807P10 - PARAISOPOLIS/SANT0 AMARO,218,1349.0,259,5,0,...,41.0,2945,1868,393,0.0,5206,dados\passageiros\2018\Passag-20180131.xls,NaN,NaN,NaN
1406,2017-01-31,PERMISSAO,AREA 8,TRANSCAP,807210 - PQ IPE/BUTANTA,618,1724.0,350,27,0,...,192.0,5691,2269,1042,0.0,9002,dados\passageiros\2018\Passag-20180131.xls,NaN,NaN,NaN
1407,2017-01-31,PERMISSAO,AREA 8,TRANSCAP,807310 - JD GUARAU/BUTANTA,428,844.0,117,15,0,...,60.0,2720,1095,550,2.0,4367,dados\passageiros\2018\Passag-20180131.xls,NaN,NaN,NaN
1408,2017-01-31,PERMISSAO,AREA 8,TRANSCAP,807810 - JD DAS PALMAS/METRO CAMPO LIMP,317,1310.0,256,25,0,...,71.0,3556,1885,1043,0.0,6484,dados\passageiros\2018\Passag-20180131.xls,NaN,NaN,NaN


In [34]:
df_unificado['data'] = pd.to_datetime(df_unificado['data'], errors='coerce')
df_unificado['route_id']=df_unificado['linha'].fillna("").apply(lambda x: x.split(" ")[0].replace("-","").title())

In [35]:
df_backup=df_unificado.copy()

In [36]:
# for c in [x for x in df_unificado.columns if 'passageiro' in x]: print(c);df_unificado[c]=df_unificado[c].fillna(0).str.replace("","0").astype('Int64') 

In [37]:
df_unificado

,data,tipo,area,empresa,linha,passageiros_dinheiro,passageiros_comum,passageiros_comum_mensal,passageiros_estudante,passageiros_estudante_mensal,...,passageiros_pagantes,passageiros_integracao_onibus,passageiros_gratuitos,passageiros_gratuitos_estudante,passageiros_total,arquivo,grupo,lote,passageiros_vale_transporte,route_id
0,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,N10511 - TERM CACHOEIRINHA/TERM LAPA,28,48.0,9,2,0,...,142,162,27,0.0,331,dados\passageiros\2019\01-FEV.xls,NaN,NaN,NaN,N10511
1,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,N10611 - T PQ D PEDRO /MET BARRA FUNDA,0,3.0,0,0,0,...,4,1,0,0.0,5,dados\passageiros\2019\01-FEV.xls,NaN,NaN,NaN,N10611
2,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,N14311 - METRO BARRA FUNDA/MORRO GRANDE,4,8.0,2,0,0,...,40,20,7,0.0,67,dados\passageiros\2019\01-FEV.xls,NaN,NaN,NaN,N14311
3,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,800001 - TERM LAPA/PCA RAMOS DE AZEVEDO,473,1290.0,208,35,0,...,3275,2541,1526,41.0,7383,dados\passageiros\2019\01-FEV.xls,NaN,NaN,NaN,800001
4,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,830001 - TERM PIRITUBA/TERM LAPA,47,169.0,24,4,0,...,410,381,187,11.0,989,dados\passageiros\2019\01-FEV.xls,NaN,NaN,NaN,830001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1405,2017-01-31,PERMISSAO,AREA 8,TRANSCAP,807P10 - PARAISOPOLIS/SANT0 AMARO,218,1349.0,259,5,0,...,2945,1868,393,0.0,5206,dados\passageiros\2018\Passag-20180131.xls,NaN,NaN,NaN,807P10
1406,2017-01-31,PERMISSAO,AREA 8,TRANSCAP,807210 - PQ IPE/BUTANTA,618,1724.0,350,27,0,...,5691,2269,1042,0.0,9002,dados\passageiros\2018\Passag-20180131.xls,NaN,NaN,NaN,807210
1407,2017-01-31,PERMISSAO,AREA 8,TRANSCAP,807310 - JD GUARAU/BUTANTA,428,844.0,117,15,0,...,2720,1095,550,2.0,4367,dados\passageiros\2018\Passag-20180131.xls,NaN,NaN,NaN,807310
1408,2017-01-31,PERMISSAO,AREA 8,TRANSCAP,807810 - JD DAS PALMAS/METRO CAMPO LIMP,317,1310.0,256,25,0,...,3556,1885,1043,0.0,6484,dados\passageiros\2018\Passag-20180131.xls,NaN,NaN,NaN,807810


In [38]:
colunas_agrupar=['passageiros_comum_mensal',
       'passageiros_estudante_mensal',
       'passageiros_vale_transporte_mensal',
       'passageiros_integracao_metro_cptm',
       'passageiros_integracao_metro_cptm_mensal',
       'passageiros_integracao_onibus', 
       'passageiros_gratuitos_estudante',
       'passageiros_comum_vt', ]

df_unificado['passageiros_vale_transporte']+= df_unificado['passageiros_comum_vt'] + df_unificado['passageiros_vale_transporte_mensal']
df_unificado['passageiros_integracao']= df_unificado['passageiros_integracao_metro_cptm'] + df_unificado['passageiros_integracao_metro_cptm_mensal'] + df_unificado['passageiros_integracao_onibus']
df_unificado['passageiros_comum']+=df_unificado['passageiros_comum_mensal']
df_unificado['passageiros_estudante']+=df_unificado['passageiros_estudante_mensal']

In [39]:
df_unificado.to_parquet('dados/passageiros_unificado.parquet', index=False)

In [40]:
df_unificado.shape

(4909893, 25)